In [1]:
import trimesh as tm
import numpy as np
import pyvista as pv
import os
import topogenesis as tg
import pickle as pk 
from ladybug.sunpath import Sunpath
from ladybug.epw import EPW

In [2]:
# create a sphere 
sphere = tm.creation.icosphere(subdivisions= 3, radius= 400.0)

# extract vertices from sphere
sphere_vertices = sphere.vertices

# keep only positive vertices
skydome = sphere_vertices[sphere_vertices[:,2] > 1.0]

# convert to array
skydome = np.array(skydome)

In [3]:
sol_path = os.path.relpath("../input/NLD_Amsterdam.062400_IWEC.epw")
epw = EPW(sol_path)

dnr = epw.direct_normal_radiation
dni = epw.direct_normal_illuminance
ghi = epw.global_horizontal_illuminance

In [4]:
sp = Sunpath(longitude=4.3571, latitude=52.0116)

sunvectors = []
hoys = []
dnrval = []
dnival = []
ghival = []

for i in range(0, 365, 14): # gives biweekly moments
    for j in range(24): # can also give a range of f.e. 12:00-14:00 hrs if needed
        k = i*24 + j
        sun = sp.calculate_sun_from_hoy(k)
        sun_vec = sun.sun_vector.to_array()
        if sun_vec[2] < 0.0:
            hoys.append(j)
            sunvectors.append(sun_vec)
            dnrval.append(dnr[k])
            dnival.append(dni[k])
            ghival.append(ghi[k])
            
# convert to numpy array
sunvectors = np.array(sunvectors)
hoys = np.array(hoys)
dnrval = np.array(dnrval)
dnival = np.array(dnival)
ghival = np.array(ghival)

# rotate vectors to correct orientation for site
rotation = 36.324
Rz = tm.transformations.rotation_matrix(np.radians(rotation), [0,0,1])
sunvectors = tm.transform_points(sunvectors, Rz)

In [5]:
# save hoys  to pk
pk.dump(hoys, open("../data/T2_hoys.pk", "wb"))

# save global horizontal illuminance to pk
pk.dump(ghival, open("../data/T2_ghival.pk", "wb"))
# save direct normal radiation to pk
pk.dump(dnrval, open("../data/T2_dnrval.pk", "wb"))
# save direct normal illuminance to pk
pk.dump(dnival, open("../data/T2_dnival.pk", "wb"))

In [6]:
# save sun vectors to pk
pk.dump(sunvectors, open("../data/T2_sunvectors.pk", "wb"))
# save skydome to pk
pk.dump(skydome, open("../data/T2_skydome.pk", "wb"))

In [7]:
p = pv.Plotter(notebook=True)

def tri_to_pv(tri_mesh):
    faces = np.pad(tri_mesh.faces, ((0, 0),(1,0)), 'constant', constant_values=3)
    pv_mesh = pv.PolyData(tri_mesh.vertices, faces)
    return pv_mesh

# fast visualization of the lattice
# envelope_lattice.fast_vis(p)

# add the sky vectors
#p.add_points(skydome, color='#0013ff')
p.add_arrows(skydome, -skydome, mag=0.1, show_scalar_bar=False)
p.add_arrows(-sunvectors*300, 300*sunvectors, mag=0.1, show_scalar_bar=False)

# add context
#p.add_mesh(tri_to_pv(context_mesh), opacity=0.1, color='lightgrey')


# plotting
p.show(use_ipyvtk=True, screenshot='T2_skydome.png')

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(1326.8095460519723, 1326.8095460519723, 1525.6196278915338),
 (0.0, 0.0, 198.81008183956146),
 (0.0, 0.0, 1.0)]